# Experiments for Higher-Order Network Statistics vs Cognitive Performance

In [1]:
# load all third part libraries
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import bct
import scipy as sp
import scipy.stats
import networkx as nx

In [2]:
# load networks
mplex2 = np.load('/Users/nick/desktop/mplex/data/mplex300_partial.npy') # normalized already
#mplex2 = mplex2.reshape(820,300,300)
#print mplex_orig.shape
mplex1 = np.load('/Users/nick/desktop/mplex/data/mplex300.npy') # shape = (820,300,300) # not normalized
mplex1_unnorm = mplex1.copy()
# load subject id's
#subjects = np.loadtxt('/Users/nick/desktop/mplex/data/subjectIDs.txt',dtype=np.int64) # shape = (820,)
# load cogntive data
cog_head, cog_vals = np.load('/Users/nick/desktop/mplex/data/demo.npy')

male_indices = np.where(cog_vals[:,3]=='M')[0]
female_indices = np.where(cog_vals[:,3]=='F')[0]

len_m = len(male_indices)
len_f = len(female_indices)

mplex1_m = mplex1[male_indices,:]
mplex2_m = mplex2[male_indices,:]
mplex1_f = mplex1[female_indices,:]
mplex2_f = mplex2[female_indices,:]

In [3]:
for i in range(len_m):
    mplex1_m[i,:,:] = (mplex1_m[i,:,:] - np.mean(mplex1_m[i,:,:]))/np.std(mplex1_m[i,:,:])
    np.fill_diagonal(mplex1_m[i,:,:],0)
for i in range(len_f):
    mplex1_f[i,:,:] = (mplex1_f[i,:,:] - np.mean(mplex1_f[i,:,:]))/np.std(mplex1_f[i,:,:])
    np.fill_diagonal(mplex1_f[i,:,:],0)

In [4]:

thresh = 3

## male

mp0 = mplex1_m.copy()
mp0[mp0 < thresh] = 0
mp0[mp0 >= thresh] = 1
for i in range(mp0.shape[0]): np.fill_diagonal(mp0[i,:,:],0)

mp1 = mplex1_m.copy()
mp1[mp1 < thresh] = 0
for i in range(mp1.shape[0]): np.fill_diagonal(mp1[i,:,:],0)
    
mp2 = mplex1_m.copy()
mp2[mp2 < 0] = 0

mp3 = mplex2_m.copy()
mp3[mp3 < thresh] = 0
mp3[mp3 >= thresh] = 1
for i in range(mp3.shape[0]): np.fill_diagonal(mp3[i,:,:],0)
    
mp4 = mplex2_m.copy()
mp4[mp4 < thresh] = 0
for i in range(mp4.shape[0]): np.fill_diagonal(mp4[i,:,:],0)
    
mp5 = mplex2_m.copy()
mp5[mp5 < 0] = 0

## female

mp6 = mplex1_f.copy()
mp6[mp6 < thresh] = 0
mp6[mp6 >= thresh] = 1
for i in range(mp6.shape[0]): np.fill_diagonal(mp6[i,:,:],0)

mp7 = mplex1_f.copy()
mp7[mp7 < thresh] = 0
for i in range(mp7.shape[0]): np.fill_diagonal(mp7[i,:,:],0)

mp8 = mplex1_f.copy()
mp8[mp8 < 0] = 0

mp9 = mplex2_f.copy()
mp9[mp9 < thresh] = 0
mp9[mp9 >= thresh] = 1
for i in range(mp9.shape[0]): np.fill_diagonal(mp9[i,:,:],0)

mp10 = mplex2_f.copy()
mp10[mp10 < thresh] = 0
for i in range(mp10.shape[0]): np.fill_diagonal(mp10[i,:,:],0)

mp11 = mplex2_f.copy()
mp11[mp11 < 0] = 0

mp_list = [mp0,mp1,mp2,mp3,mp4,mp5,mp6,mp7,mp8,mp9,mp10,mp11]

def z_norm(x):
    """
    Normalize values to be z-scores
    """
    for i in range(x.shape[0]):
        x[i,:,:] = (x[i,:,:] - np.mean(x[i,:,:])) / np.std(x[i,:,:])
    x[x<0] = 0
    return x

mp13 = mplex2_f.copy()
mp13[mp13 < 4.5] = 0
mp13[mp13 >= 4.5] = 1
for i in range(mp13.shape[0]): np.fill_diagonal(mp13[i,:,:],0)

mp14 = mplex2_f.copy()
mp14[mp14 < 6] = 0
mp14[mp14 >= 6] = 1
for i in range(mp14.shape[0]): np.fill_diagonal(mp14[i,:,:],0)
    
mp15 = mplex2_f.copy()
mp15[mp15 < 9] = 0
mp15[mp15 >= 9] = 1
for i in range(mp15.shape[0]): np.fill_diagonal(mp15[i,:,:],0)

mp16 = mplex2_m.copy()
mp16[mp16 < 4.5] = 0
mp16[mp16 >= 4.5] = 1
for i in range(mp16.shape[0]): np.fill_diagonal(mp16[i,:,:],0)

mp17 = mplex2_m.copy()
mp17[mp17 < 6] = 0
mp17[mp17 >= 6] = 1
for i in range(mp17.shape[0]): np.fill_diagonal(mp17[i,:,:],0)
    
mp18 = mplex2_m.copy()
mp18[mp18 < 9] = 0
mp18[mp18 >= 9] = 1
for i in range(mp18.shape[0]): np.fill_diagonal(mp18[i,:,:],0)




mp_list_extended = [mp0,mp1,mp2,mp3,mp4,mp5,mp6,mp7,mp8,mp9,mp10,mp11,mp13,mp14,mp15,mp16,mp17,mp18]


Mplex Code:
    - mp1 : full correlation, normalized, binary thresholded
    - mp2 : full correlation, normalized, thresholded
    - mp3 : full correlation, normalized, thresholded at 0
    - mp4 : partial correlation, normalized, binary thresholded
    - mp5 : partial correaltion, normalized, thresholded
    - mp6 : partial correlation, normalized, thresholded at 0


Cognitive data pre-processing and function for testing correlation between cognitive data and network metrics:

In [5]:
def str_to_int(x):
    """
    Convert array of strings to their integer indices
    """
    try:
        y = x.astype(np.float32)
    except ValueError:
        x = x.astype('str')
        unique_vals = np.unique(x)
        y = np.zeros(len(x),dtype=np.int32)
        for i, val in enumerate(x):
            y[i] = np.where(val==unique_vals)[0][0]
    return y

def test_corr(cog_vec, net_vec):
    """
    Test correlation between cognitive values (cog_vec)
    and network statistic values (net_vec). Each individual
    should have a single value for each - i.e. len = 820
    """
    # create mask
    if net_vec.shape[0] == 366:
        cog_vec = cog_vec[male_indices]
    else:
        cog_vec = cog_vec[female_indices]
    midx = np.where(cog_vec=='nan')
    mask = np.ones(net_vec.shape[0],dtype='bool')
    mask[midx] = 0
    # convert to int
    cog_vec = str_to_int(cog_vec)
    # mask
    cog_vec = cog_vec[mask]
    net_vec = net_vec[mask]
    # normalize cog vals
    cog_vec = (cog_vec - np.mean(cog_vec)) / (np.std(cog_vec)+1e-7)
    net_vec = (net_vec - np.mean(net_vec)) / (np.std(net_vec)+1e-7)
    # test
    r,p = sp.stats.pearsonr(cog_vec,net_vec)
    return r,p

## 1. Centrality

Here, we test full (mplex1) and partial (mplex2) correlation networks using various centrality measures. We perform this analysis for binary, thresholded normalized, and unthresholded normalized versions of each network.

For each metric that returns a vector of values for each individual network, we will take the variance, the max, and the mean.

### 1A. Node Betweenness

In [6]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn = bct.betweenness_bin(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250


//anaconda/lib/python2.7/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


MALE:  0  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.31634907721 Pval:  0.0641052608373
MALE:  0  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.407509701391 Pval:  0.0151062651787
MALE:  0  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.28139538979 Pval:  0.112646159842
Mplex :  1
Net Idx :  0
Net Idx :  250
MALE:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.290962550684 Pval:  0.0899326873097
MALE:  1  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.388623588882 Pval:  0.0210463345807
MALE:  1  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.26571147912 Pval:  0.122888318919
MALE:  1  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.28360871238 Pval:  0.109725982528
Mplex :  2
Net Idx :  0
Net Idx :  250
MALE:  2  TRANS:  0  Col:  SSAGA_Mental_Compl  Corr:  -0.259175065083 Pval:  4.96882209619e-07
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  0  Col:  MEGsession_Label  Cor

### 1B. PageRank

In [7]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn = bct.pagerank_centrality(net,0.85)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
MALE:  0  TRANS:  2  Col:  fMRI_Soc_PctCompl  Corr:  -0.287396542376 Pval:  2.16904850491e-08
MALE:  0  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  -0.34606025686 Pval:  0.0485246756322
Mplex :  1
Net Idx :  0
Net Idx :  250
MALE:  1  TRANS:  2  Col:  fMRI_Soc_PctCompl  Corr:  -0.293164198902 Pval:  1.09422235738e-08
Mplex :  2
Net Idx :  0
Net Idx :  250
MALE:  2  TRANS:  1  Col:  MEGsession_Label  Corr:  0.262964497219 Pval:  0.0679244071337
MALE:  2  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.254732180741 Pval:  0.139736931398
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  0  Col:  MEGsession_Label  Corr:  0.26142778782 Pval:  0.0696082445355
MALE:  3  TRANS:  2  Col:  MEGsession_Label  Corr:  0.252600632432 Pval:  0.0799323904645
MALE:  3  TRANS:  1  Col:  Alpha_Peak  Corr:  0.313199212375 Pval:  0.0301913921747
MALE:  3  TRANS:  2  Col:  FS_L_LatVent_Vol  Corr:  0.341766938908 Pval:  1.82449311785e-11

### 1C. Eigenvector Centrality

In [8]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn = bct.eigenvector_centrality_und(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
MALE:  0  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Random  Corr:  0.475394699937 Pval:  0.00390025006287
Mplex :  1
Net Idx :  0
Net Idx :  250
MALE:  1  TRANS:  2  Col:  MEGsession_Label  Corr:  -0.326000597388 Pval:  0.0222579458629
MALE:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  0.272191851619 Pval:  0.113681648644
MALE:  1  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.289339480025 Pval:  0.0918217592895
MALE:  1  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Random  Corr:  0.367406301916 Pval:  0.0299127990467
MALE:  1  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.467545852537 Pval:  0.00607849319164
Mplex :  2
Net Idx :  0
Net Idx :  250
MALE:  2  TRANS:  0  Col:  MEGsession_Label  Corr:  0.323372559364 Pval:  0.0234240919045
MALE:  2  TRANS:  1  Col:  MEGsession_Label  Corr:  -0.331060371505 Pval:  0.0201499998865
MALE:  2  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.33664556

## 2. Clustering

### 2A. Clustering Coefficienct

In [9]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        ccf = bct.clustering_coef_bu(net)
        nb[0,net_idx] = np.var(ccf)
        nb[1,net_idx] = np.mean(ccf)
        nb[2,net_idx] = np.max(ccf)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
MALE:  0  TRANS:  0  Col:  Beta_Peak  Corr:  0.256614407102 Pval:  0.0750979083231
MALE:  0  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.303851338974 Pval:  0.0855958840193
MALE:  0  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.276430039272 Pval:  0.119408604869
Mplex :  1
Net Idx :  0
Net Idx :  250
Mplex :  2
Net Idx :  0
Net Idx :  250
MALE:  2  TRANS:  0  Col:  MEGsession_Label  Corr:  0.267966112165 Pval:  0.0626673052407
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  2  Col:  MEGsession_Label  Corr:  0.331137811687 Pval:  0.0201190955539
MALE:  3  TRANS:  0  Col:  Alpha_Peak  Corr:  0.276236269179 Pval:  0.0573626974908
MALE:  3  TRANS:  0  Col:  Social_Task_Random_Median_RT_TOM  Corr:  -0.311478129381 Pval:  0.00773715553875
Mplex :  4
Net Idx :  0
Net Idx :  250
MALE:  4  TRANS:  0  Col:  Social_Task_Random_Median_RT_TOM  Corr:  -0.290523029468 Pval:  0.0132984428957
Mplex :  5
Net Idx :  0
Net Idx 

### 2B. Transitivity

In [10]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn = bct.transitivity_bu(net)
        else:
            btwn = bct.transitivity_wu(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
MALE:  0  TRANS:  1  Col:  MEGsession_Label  Corr:  0.267109876336 Pval:  0.0635434199124
MALE:  0  TRANS:  2  Col:  MEGsession_Label  Corr:  0.267109876336 Pval:  0.0635434199124
Mplex :  1
Net Idx :  0
Net Idx :  250
Mplex :  2
Net Idx :  0
Net Idx :  250
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.306313271039 Pval:  0.082964496222
MALE:  3  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.306313271039 Pval:  0.082964496222
Mplex :  4
Net Idx :  0
Net Idx :  250
MALE:  4  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.289813187155 Pval:  0.101843184304
MALE:  4  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.289813187155 Pval:  0.101843184304
Mplex :  5
Net Idx :  0
Net Idx :  250
Mplex :  6
Net Idx :  0
Net Idx :  250
Mplex :  7
Net Idx :  0
Net Idx :  250
Mplex :  8
Net Idx :  0
Net Idx :  250
FEMALE:  8  TRANS:  1  Col:  Alpha_Peak  Corr:  -

## 3. Core

### 3A. Assortativity

In [11]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn = bct.assortativity_bin(net,0)
        else:
            btwn = bct.assortativity_wei(net,0)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Mplex :  1
Net Idx :  0
Net Idx :  250
MALE:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.262178751802 Pval:  0.128135448506
MALE:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  -0.252570015936 Pval:  0.156170047133
Mplex :  2
Net Idx :  0
Net Idx :  250
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  0  Col:  fMRI_Soc_PctCompl  Corr:  -0.31031561163 Pval:  1.30488979514e-09
MALE:  3  TRANS:  0  Col:  MEGsession_Scans  Corr:  0.277702563519 Pval:  0.0533690221715
Mplex :  4
Net Idx :  0
Net Idx :  250
MALE:  4  TRANS:  0  Col:  fMRI_Soc_PctCompl  Corr:  -0.294727373914 Pval:  9.06603540169e-09
Mplex :  5
Net Idx :  0
Net Idx :  250
MALE:  5  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.336579519775 Pval:  0.0480389938328
Mplex :  6
Net Idx :  0
Net Idx :  250
FEMALE:  6  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.263258190268 Pval:  0.0198730713742
Mplex :  7
Net Idx :  0

### 3B. Rich Club --  GOOD RESULTS HERE, DON'T ALTER

In [12]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((20,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn,_,_ = bct.rich_club_bu(net,20)
        else:
            btwn = bct.rich_club_wu(net,20)
        for ii in range(20):
            nb[ii,net_idx] = btwn[ii]
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0


//anaconda/lib/python2.7/site-packages/bctpy-0.4.1-py2.7.egg/bct/algorithms/core.py:354: RuntimeWarning: invalid value encountered in double_scalars
  R[k] = Ek[k] / (Nk[k] * (Nk[k] - 1))


Net Idx :  250
MALE:  0  TRANS:  0  Col:  MEGsession_Label  Corr:  0.326137033155 Pval:  0.0221987633056
MALE:  0  TRANS:  1  Col:  MEGsession_Label  Corr:  0.255122250322 Pval:  0.0768674473247
MALE:  0  TRANS:  2  Col:  MEGsession_Label  Corr:  0.283540324893 Pval:  0.0483511821907
MALE:  0  TRANS:  3  Col:  MEGsession_Label  Corr:  0.266864061596 Pval:  0.063796742321
Mplex :  1
Net Idx :  0


//anaconda/lib/python2.7/site-packages/bctpy-0.4.1-py2.7.egg/bct/algorithms/core.py:448: RuntimeWarning: invalid value encountered in double_scalars
  Rw[k] = Wr / np.sum(wrank_r)


Net Idx :  250
MALE:  1  TRANS:  0  Col:  Social_Task_Random_Median_RT_TOM  Corr:  -0.257780345808 Pval:  0.0288047345291
MALE:  1  TRANS:  7  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.332889349242 Pval:  0.0506998434913
MALE:  1  TRANS:  9  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.311614762683 Pval:  0.0684157996945
MALE:  1  TRANS:  10  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.276847551708 Pval:  0.107394132955
MALE:  1  TRANS:  11  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.2704882698 Pval:  0.116050105116
MALE:  1  TRANS:  12  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.282048898589 Pval:  0.10068432951
MALE:  1  TRANS:  3  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  -0.256452671082 Pval:  0.149690689007
Mplex :  2
Net Idx :  0
Net Idx :  250
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  16  Col:  MEGsession_Label  Corr:  0.284889601401 Pval:  0.047247758748
MALE:  3  TRANS:  17  Col:  MEGsession_Label  Corr:  0.283947943515 Pval

In [13]:
# extended mplex list
for mp_idx in range(5,len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((20,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn,_,_ = bct.rich_club_bu(net,20)
        else:
            btwn = bct.rich_club_wu(net,20)
        for ii in range(20):
            nb[ii,net_idx] = btwn[ii]
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  5
Net Idx :  0
Net Idx :  250
Mplex :  6
Net Idx :  0
Net Idx :  250
Mplex :  7
Net Idx :  0
Net Idx :  250
FEMALE:  7  TRANS:  6  Col:  MEGsession_Label  Corr:  -0.306785783704 Pval:  0.0403849188576
FEMALE:  7  TRANS:  7  Col:  MEGsession_Label  Corr:  -0.302323270889 Pval:  0.0435409896747
FEMALE:  7  TRANS:  9  Col:  Alpha_Peak  Corr:  0.339665706325 Pval:  0.022444203156
FEMALE:  7  TRANS:  10  Col:  Alpha_Peak  Corr:  0.290641959243 Pval:  0.0527624261525
FEMALE:  7  TRANS:  11  Col:  Alpha_Peak  Corr:  0.305615186929 Pval:  0.0411941243895
FEMALE:  7  TRANS:  12  Col:  Alpha_Peak  Corr:  0.327120686399 Pval:  0.0282800430226
FEMALE:  7  TRANS:  13  Col:  Alpha_Peak  Corr:  0.340334445964 Pval:  0.0221638492081
FEMALE:  7  TRANS:  14  Col:  Alpha_Peak  Corr:  0.306161476182 Pval:  0.0408148544958
FEMALE:  7  TRANS:  9  Col:  Beta_Peak  Corr:  0.329002736008 Pval:  0.0292100877514
FEMALE:  7  TRANS:  10  Col:  Beta_Peak  Corr:  0.343218449893 Pval:  0.0225551486038
FEMALE

## 4. Degree

In [14]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn = bct.degrees_und(net)
        else:
            btwn = bct.strengths_und(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)  
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
MALE:  0  TRANS:  0  Col:  MEGsession_Label  Corr:  0.378205358346 Pval:  0.0073749623799
MALE:  0  TRANS:  2  Col:  MEGsession_Label  Corr:  0.339206646214 Pval:  0.0171109414634
Mplex :  1
Net Idx :  0
Net Idx :  250
MALE:  1  TRANS:  0  Col:  MEGsession_Label  Corr:  0.283857922422 Pval:  0.0480895881704
MALE:  1  TRANS:  2  Col:  MEGsession_Label  Corr:  0.267167082445 Pval:  0.0634845821052
Mplex :  2
Net Idx :  0
Net Idx :  250
MALE:  2  TRANS:  0  Col:  Social_Task_Random_Median_RT_TOM  Corr:  0.258160360741 Pval:  0.0285615947562
MALE:  2  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.312378045334 Pval:  0.0767480558511
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Random  Corr:  0.326731135981 Pval:  0.0554012563228
Mplex :  4
Net Idx :  0
Net Idx :  250
MALE:  4  TRANS:  2  Col:  FS_LCort_GM_Vol  Corr:  0.260268747365 Pval:  4.42994721367e-07
MALE:  4  TRANS:  2  Col:  FS_RCort_

## 5. Distance

### 5A. Characteristic Path Length

In [15]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn,_,_,_,_ = bct.charpath(net+1e-7)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Mplex :  1
Net Idx :  0
Net Idx :  250
Mplex :  2
Net Idx :  0
Net Idx :  250
Mplex :  3
Net Idx :  0
Net Idx :  250
Mplex :  4
Net Idx :  0
Net Idx :  250
Mplex :  5
Net Idx :  0
Net Idx :  250
Mplex :  6
Net Idx :  0
Net Idx :  250
Mplex :  7
Net Idx :  0
Net Idx :  250
Mplex :  8
Net Idx :  0
Net Idx :  250
FEMALE:  8  TRANS:  0  Col:  Alpha_Peak  Corr:  -0.283448091447 Pval:  0.0591842955945
FEMALE:  8  TRANS:  0  Col:  Beta_Peak  Corr:  -0.301494440617 Pval:  0.0467164293649
Mplex :  9
Net Idx :  0
Net Idx :  250
FEMALE:  9  TRANS:  0  Col:  fMRI_3T_ReconVrs  Corr:  0.282308050927 Pval:  9.54463751004e-10
FEMALE:  9  TRANS:  0  Col:  FS_IntraCranial_Vol  Corr:  0.315990580039 Pval:  5.81683751005e-12
FEMALE:  9  TRANS:  0  Col:  FS_BrainSeg_Vol  Corr:  0.298181811433 Pval:  9.36735789344e-11
FEMALE:  9  TRANS:  0  Col:  FS_BrainSeg_Vol_No_Vent  Corr:  0.297859711832 Pval:  9.83331207242e-11
FEMALE:  9  TRANS:  0  Col:  FS_BrainSeg_Vol_No_Vent

### 5B. Efficiency from CharPath

In [16]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        _,btwn,_,_,_ = bct.charpath(net+1e-7)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.25:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Mplex :  1
Net Idx :  0
Net Idx :  250
Mplex :  2
Net Idx :  0
Net Idx :  250
Mplex :  3
Net Idx :  0
Net Idx :  250
Mplex :  4
Net Idx :  0
Net Idx :  250
Mplex :  5
Net Idx :  0
Net Idx :  250
MALE:  5  TRANS:  0  Col:  Release  Corr:  -0.455320636903 Pval:  3.93077051493e-20
MALE:  5  TRANS:  0  Col:  Acquisition  Corr:  -0.364972165273 Pval:  5.65246494544e-13
MALE:  5  TRANS:  0  Col:  fMRI_3T_ReconVrs  Corr:  -0.590585358297 Pval:  8.80489451233e-36
Mplex :  6
Net Idx :  0
Net Idx :  250
Mplex :  7
Net Idx :  0
Net Idx :  250
Mplex :  8
Net Idx :  0
Net Idx :  250
FEMALE:  8  TRANS:  0  Col:  Alpha_Peak  Corr:  0.270632477185 Pval:  0.072162600961
FEMALE:  8  TRANS:  0  Col:  Beta_Peak  Corr:  0.340782882005 Pval:  0.0235954601629
Mplex :  9
Net Idx :  0
Net Idx :  250
FEMALE:  9  TRANS:  0  Col:  fMRI_3T_ReconVrs  Corr:  -0.282308050927 Pval:  9.54463751004e-10
FEMALE:  9  TRANS:  0  Col:  FS_IntraCranial_Vol  Corr:  -0.315990580039 Pval:  

### 5C. Global Efficiency

In [17]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn,_ = bct.efficiency_bin(net)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.20:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
MALE:  0  TRANS:  0  Col:  Alpha_Peak  Corr:  -0.200689139359 Pval:  0.17139797626
MALE:  0  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.247683314278 Pval:  0.151412067599
Mplex :  1
Net Idx :  0
Net Idx :  250
MALE:  1  TRANS:  0  Col:  Alpha_Peak  Corr:  -0.200689139359 Pval:  0.17139797626
MALE:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.247683314278 Pval:  0.151412067599
Mplex :  2
Net Idx :  0
Net Idx :  250
MALE:  2  TRANS:  0  Col:  Social_Task_Median_RT_Unsure  Corr:  -0.200887071731 Pval:  0.0146970794716
MALE:  2  TRANS:  0  Col:  Social_Task_Random_Median_RT_Unsure  Corr:  -0.234381360909 Pval:  0.00799503739333
Mplex :  3
Net Idx :  0
Net Idx :  250
MALE:  3  TRANS:  0  Col:  FS_4thVent_Vol  Corr:  0.215159340818 Pval:  3.31204765133e-05
MALE:  3  TRANS:  0  Col:  FS_L_Superiorfrontal_Thck  Corr:  0.203906136458 Pval:  8.53120520928e-05
Mplex :  4
Net Idx :  0
Net Idx :  250
MALE:  4  TRANS:  0  C

### 5D. Local Efficiency - IMPORTANT RESULTS IN MPLEX11

In [18]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((6,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        _,btwn = bct.efficiency_bin(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.std(btwn)
        nb[3,net_idx] = np.max(btwn)
        nb[4,net_idx] = np.min(btwn)
        nb[5,net_idx] = np.max(btwn) - np.min(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.20:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
MALE:  0  TRANS:  1  Col:  Alpha_Peak  Corr:  -0.200689139359 Pval:  0.17139797626
MALE:  0  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.247683314278 Pval:  0.151412067599
Mplex :  1
Net Idx :  0
Net Idx :  250
MALE:  1  TRANS:  1  Col:  Alpha_Peak  Corr:  -0.200689139359 Pval:  0.17139797626
MALE:  1  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.247683314278 Pval:  0.151412067599
Mplex :  2
Net Idx :  0
Net Idx :  250
MALE:  2  TRANS:  0  Col:  Social_Task_Median_RT_Unsure  Corr:  -0.205025745675 Pval:  0.0127345974707
MALE:  2  TRANS:  1  Col:  Social_Task_Median_RT_Unsure  Corr:  -0.200887071731 Pval:  0.0146970794716
MALE:  2  TRANS:  2  Col:  Social_Task_Median_RT_Unsure  Corr:  -0.205076918404 Pval:  0.0127118436061
MALE:  2  TRANS:  0  Col:  Social_Task_Random_Median_RT_Unsure  Corr:  -0.241895438263 Pval:  0.0061461010597
MALE:  2  TRANS:  1  Col:  Social_Task_Random_Median_RT_Unsure  Corr:  -0.234381360909

### 5E. Eccentricity

In [19]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((6,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        _,_,btwn,_,_ = bct.charpath(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.std(btwn)
        nb[3,net_idx] = np.max(btwn)
        nb[4,net_idx] = np.min(btwn)
        nb[5,net_idx] = np.max(btwn) - np.min(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.2:
                if mp_idx < 6:
                    print 'MALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval
                else:
                    print 'FEMALE: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0


//anaconda/lib/python2.7/site-packages/bctpy-0.4.1-py2.7.egg/bct/algorithms/distance.py:159: RuntimeWarning: divide by zero encountered in true_divide
  D = 1 / D  # invert distance


Net Idx :  250
MALE:  0  TRANS:  2  Col:  MEGsession_Label  Corr:  -0.200411890412 Pval:  0.167364570512
MALE:  0  TRANS:  0  Col:  Alpha_Peak  Corr:  -0.224668624618 Pval:  0.124729667984
MALE:  0  TRANS:  1  Col:  Alpha_Peak  Corr:  -0.238422782383 Pval:  0.102694268352
MALE:  0  TRANS:  2  Col:  Alpha_Peak  Corr:  -0.223160971521 Pval:  0.127346618757
MALE:  0  TRANS:  0  Col:  FS_L_Cerebellum_Cort_Vol  Corr:  0.219419359777 Pval:  2.28448315341e-05
MALE:  0  TRANS:  1  Col:  FS_L_Cerebellum_Cort_Vol  Corr:  0.231674024724 Pval:  7.53122072215e-06
MALE:  0  TRANS:  2  Col:  FS_L_Cerebellum_Cort_Vol  Corr:  0.217722483515 Pval:  2.6510802663e-05
MALE:  0  TRANS:  0  Col:  FS_L_Cerebellum_Cort_Vol.1  Corr:  0.213179232213 Pval:  3.92639224647e-05
MALE:  0  TRANS:  1  Col:  FS_L_Cerebellum_Cort_Vol.1  Corr:  0.224882298897 Pval:  1.403628654e-05
MALE:  0  TRANS:  2  Col:  FS_L_Cerebellum_Cort_Vol.1  Corr:  0.21140462457 Pval:  4.56710645702e-05
MALE:  0  TRANS:  0  Col:  Social_Task_TO

## 7. Motifs

### 7A. Binary Structural 3-node Motifs

### 7B. Weighted Structural 3-node Motifs